In [13]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

In [14]:
dataset = pd.read_csv('dataset/winequality-red.csv', delimiter=';')

In [15]:
dataset

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [16]:
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

In [17]:
y

array([5, 5, 5, ..., 6, 5, 6], dtype=int64)

In [18]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)

In [19]:
y = tf.keras.utils.to_categorical(y)

In [20]:
y

array([[0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       ...,
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.]], dtype=float32)

In [21]:
len(y[0])

6

In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [23]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [24]:
ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units = 128, activation = 'relu'))
ann.add(tf.keras.layers.Dropout(0.3))
ann.add(tf.keras.layers.Dense(units = 128, activation = 'relu'))
ann.add(tf.keras.layers.Dense(units = 6, activation = 'softmax'))
ann.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [25]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 300, validation_data = (X_test, y_test))

Epoch 1/300
40/40 [==============================] - 1s 5ms/step - loss: 1.3035 - accuracy: 0.4855 - val_loss: 1.0603 - val_accuracy: 0.5750
Epoch 2/300
40/40 [==============================] - 0s 3ms/step - loss: 1.0182 - accuracy: 0.5754 - val_loss: 0.9971 - val_accuracy: 0.5750
Epoch 3/300
40/40 [==============================] - 0s 3ms/step - loss: 0.9712 - accuracy: 0.5903 - val_loss: 0.9864 - val_accuracy: 0.6000
Epoch 4/300
40/40 [==============================] - 0s 3ms/step - loss: 0.9494 - accuracy: 0.6052 - val_loss: 0.9774 - val_accuracy: 0.6000
Epoch 5/300
40/40 [==============================] - 0s 3ms/step - loss: 0.9374 - accuracy: 0.6059 - val_loss: 0.9709 - val_accuracy: 0.5906
Epoch 6/300
40/40 [==============================] - 0s 3ms/step - loss: 0.9226 - accuracy: 0.6231 - val_loss: 0.9735 - val_accuracy: 0.5781
Epoch 7/300
40/40 [==============================] - 0s 3ms/step - loss: 0.9000 - accuracy: 0.6122 - val_loss: 0.9698 - val_accuracy: 0.5875
Epoch 8/300
4

In [26]:
y_pred = ann.predict(X_test)
print(np.round(y_pred,3))

[[0.    0.    0.008 0.984 0.008 0.   ]
 [0.    0.001 0.829 0.17  0.    0.   ]
 [0.    0.088 0.044 0.868 0.    0.   ]
 ...
 [0.    0.    0.97  0.03  0.001 0.   ]
 [0.    0.    0.98  0.02  0.    0.   ]
 [0.    0.018 0.732 0.25  0.    0.   ]]


In [27]:
ann.evaluate(X_test,y_test)

10/10 [==============================] - 0s 1ms/step - loss: 1.6522 - accuracy: 0.6719


[1.65220046043396, 0.671875]

In [28]:
print(tf.math.confusion_matrix(np.argmax(y_test, axis=1),np.argmax(y_pred, axis=1)))

tf.Tensor(
[[ 0  1  1  0  0  0]
 [ 0  0  7  4  0  0]
 [ 0  3 94 33  3  0]
 [ 0  2 28 94  5  1]
 [ 0  0  0 14 26  0]
 [ 0  0  1  1  1  1]], shape=(6, 6), dtype=int32)
